# Quantify transports between Rockall Trough and Faroe Shetland Channel from either end.

- Lagrangian particle tracking in VIKING20X model
- Particles tracked backwards for 2 years from FS Channel and forward for 2 years from RT
- Advection only
- Transport calculations based on the idea of particle track representing a streamtube with constant along-tube transport everywhere
- Store T and S to examine along-track T-S properties

This is one in a series of notebooks exploring the tracking output. Here we
- load lagrangian trajectory data that has been written by OceanParcels from particle tracking in Viking20x model (currently using monthly mean fields for speed)
    - particles tracked from a section across RT and FS
    - random release points along section
    - tracked forward from 6 RT for 2 years
    - tracked backwards from FS for 2 years
    - focus on 2014
- use this to plot timeseries of transport through the source sections

Transports associated with different pathways are estimated by assuming that each particle crossing FS Channel has an associated volume transport, which it maintains throughout its life, given by the 
                velocity across the section * total cross-sectional area / number of particles
So like a streamtube...    

I suspect that most of the volume transport is coming through quite quickly following the eastern boundary, but may be obscured in track plots and animations by all the eddying motions. Trying to check this with total transport and transport with longitude time series across the section.

## Technical preamble

In [ ]:
%matplotlib inline

import matplotlib.colors as colors
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import xarray as xr
from datetime import datetime, timedelta
import seaborn as sns
from matplotlib.colors import ListedColormap
import cmocean as co

sns.set_palette("colorblind")

## Parameters

In [ ]:
# Project path
project_path = Path.cwd() 
project_path = project_path.resolve()

# Parcels track data file
path_data_tracks = Path('data_output') 
fname_FS = f'tracks_FS_backward_2014*D730*.nc'
files_FS = list(sorted((project_path / path_data_tracks).glob(fname_FS)))

fname_RT = f'tracks_RT_forwards_2014*D730*.nc'
files_RT = list(sorted((project_path / path_data_tracks).glob(fname_RT)))


# model mask file
data_path = Path("/data/iAtlantic/")
experiment_name = "VIKING20X.L46-KKG36107B"
mesh_mask_file = data_path / "mask" / experiment_name / "1_mesh_mask.nc"

In [ ]:
#velocity conversions in FS Channel

lat2metres_FS = 60.0*1852.0
lon2metres_FS = 60.0*1852.0*np.cos(np.radians(61.25))

sealandratio_FS = 5591.0/20000.0
sectionAreaEW_FS = 5.7 * 1500.0 * sealandratio_FS * lon2metres_FS  # very approx. mean depth x length between slopes.
sectionAreaNS_FS = 1.87 * 1500.0 * sealandratio_FS * lat2metres_FS

#velocity conversions in Rockall Trough

lat2metres_RT = 60.0*1852.0
lon2metres_RT = 60.0*1852.0*np.cos(np.radians(57.5))

sealandratio_RT = 6007.0/20000.0
sectionAreaEW_RT = 8.0 * 2500.0 * sealandratio_RT * lon2metres_RT  # very approx. mean depth x length between slopes.
sectionAreaNS_RT = 0


In [ ]:
def plot_tracks(x,y,z,title):
    plt.figure(figsize = (12,9))

    # plot tracks as scatter plots

    plt.scatter(
        x
        ,y
        ,3
        ,z
#        ,vmin = tvmin, vmax = tvmax
        ,cmap = co.cm.delta
    #    ,alpha=0.3
    )
    plt.colorbar(label = "time [stupid units. range 4 years, all cross IFR at 2 years]");

    # plot depth contours and land boundary from model bathymetry
    depth.isel(y=slice(1700, 2499), x=slice(1300, 2404)).plot.contour(
        x="nav_lon", y="nav_lat", colors = 'grey', levels = [200,800,1500,2000,2500,3500]
    );
    depth.isel(y=slice(1700, 2499), x=slice(1300, 2404)).plot.contour(
        x="nav_lon", y="nav_lat", colors = 'k', levels = [1]
    );
    # plt.savefig('upstream.png')
    plt.ylim(45,70)
    plt.xlim(-30,10)
    plt.title(title)
    plt.show()
    return

In [ ]:
def apply_through_area(ds, min_lon, max_lon, min_lat, max_lat, min_time, max_time, criterion_name):
    '''Apply an area crossing criterion.
    
    Larvae in ds selected if they pass through given area.
    '''
    # particles are selected if they pass through given area.
    particle_out =  (
                    (ds.lon > min_lon) * 
                    (ds.lon < max_lon) *
                    (ds.lat > min_lat) * 
                    (ds.lat < max_lat) *
                    (ds.time > min_time) *
                    (ds.time < max_time)
                    )
                    
#     particle_out = (ds.temp < max_lat)
    

    # last place in array determines if particle went through area  
    is_thru = (particle_out.cumsum("obs") != 0)
    
#     print(particle_out.max())
    
    # add data to original ds
    ds[criterion_name] = is_thru
    
    return ds

In [ ]:
def apply_left_of_line(ds, lon_1, lon_2, lat_1, lat_2, min_time, max_time, criterion_name):
    '''Apply an area crossing criterion.
    
    Larvae in ds selected if they pass through given area.
    '''
    # particles are selected if they pass through given area.
    particle_out =  ((((lon_2 -lon_1) * (ds.lat - lat_1) - 
                     (ds.lon - lon_1) * (lat_2 - lat_1))
                     > 0.0) *
                    (ds.time > min_time) *
                    (ds.time < max_time))
                    
#     particle_out = (ds.temp < max_lat)
    

    # last place in array determines if particle went through area  
    is_thru = (particle_out.cumsum("obs") != 0)
    
#     print(particle_out.max())
    
    # add data to original ds
    ds[criterion_name] = is_thru
    
    return ds

In [ ]:
def mask_tracks_by_source(ds_full):
    # comes through Rockall Trough and W Scotland shelf
    ds_area = apply_through_area(ds_full,-13.5,-6.5,57.25,57.75,
                                 np.datetime64('2010-06-15'),np.datetime64('2020-06-15'),
                                 'F_RT_WSS')    
    # comes/goes northward west of Rockall Bank
    ds_area = apply_through_area(ds_area,-60,-13.5,57.25,57.75,
                                 np.datetime64('2010-06-15'),np.datetime64('2020-06-15'),
                                 'FT_WRB')  
    # comes/goes clockwise round Iceland
    ds_area = apply_through_area(ds_area,-22,-20,65,70,np.datetime64('2010-06-15'),
                                 np.datetime64('2020-06-15'),
                                 'FT_WIceland')  
    # comes/goes round Faroe
    ds_area = apply_through_area(ds_area,-20,-6,68.5,70,
                                 np.datetime64('2010-06-15'),np.datetime64('2020-06-15'),
                                 'FT_GreenSea')  
    # pass north of Faroe
    ds_area = apply_through_area(ds_area,-6.5,-5.5,62.3,64.5,
                                 np.datetime64('2010-06-15'),np.datetime64('2016-06-15'),
                                 'FT_NFaroes')    
    return ds_area

In [ ]:
def mask_tracks_by_destination(ds_full):
    # goes to arctic
    ds_area = apply_left_of_line(ds_full,-7,-1.3,62.2,60.33,
                                 np.datetime64('2010-06-15'),np.datetime64('2020-06-15'),
                                 'T_FS')
    # comes/goes northward west of Rockall Bank
    ds_area = apply_through_area(ds_area,-60,-13.5,57.25,57.75,
                                 np.datetime64('2010-06-15'),np.datetime64('2020-06-15'),
                                 'FT_WRB')  
    # comes/goes clockwise round Iceland
    ds_area = apply_through_area(ds_area,-22,-20,65,70,np.datetime64('2010-06-15'),
                                 np.datetime64('2020-06-15'),
                                 'FT_WIceland')  
    # comes/goes round Faroe
    ds_area = apply_through_area(ds_area,-20,-6,68.5,70,
                                 np.datetime64('2010-06-15'),np.datetime64('2020-06-15'),
                                 'FT_GreenSea')  
    # pass north of Faroe
    ds_area = apply_through_area(ds_area,-6.5,-5.5,62.3,64.5,
                                 np.datetime64('2010-06-15'),np.datetime64('2016-06-15'),
                                 'FT_NFaroes')    
    return ds_area

## Load data

### Load model mesh mask

In [ ]:
mesh_mask = xr.open_dataset(mesh_mask_file)
mesh_mask = mesh_mask.squeeze()
mesh_mask = mesh_mask.set_coords(["nav_lon", "nav_lat", "nav_lev"])

bathy = mesh_mask.mbathy.rename("number of water filled points")

depth = (mesh_mask.e3t_0 * mesh_mask.tmask).sum("z")
# display(mesh_mask)

### Load tracks

In [ ]:
ds_FS = []
for i,file in enumerate(files_FS):
    ds_FS.append(xr.open_dataset(file))

ds_FS_all = xr.concat(ds_FS,dim='traj')  

ds_RT = []
for i,file in enumerate(files_RT):
    ds_RT.append(xr.open_dataset(file))

ds_RT_all = xr.concat(ds_RT,dim='traj')  

In [ ]:
display(ds_FS_all)
display(ds_RT_all)
    

## Calculate some basic parameters from data for later

In [ ]:
nParticles_FS = ds_FS_all.sizes['traj']
nParticles_RT = ds_RT_all.sizes['traj']


### Find subsets of trajectories passing through different areas, either upstream or downstream

This just tests whether particles were ever in an area, much easier than testing line crossings.

In [ ]:
ds_FS_all_routes = mask_tracks_by_source(ds_FS_all)
ds_RT_all_routes = mask_tracks_by_destination(ds_RT_all)


## Plot tracks and time series for some cases

### Tracks ending at FSC coming from south through Rockall Trough

In [ ]:
ds_slice = ds_FS_all_routes.isel(traj=slice(0,-1,100))
lon =    (ds_slice.where(ds_slice.isel(obs=-1).FT_NFaroes == False)
                .where(ds_slice.isel(obs=-1).FT_WRB == False)
                .where(ds_slice.isel(obs=-1).F_RT_WSS).lon.data.flatten())
lat =    (ds_slice.where(ds_slice.isel(obs=-1).FT_NFaroes == False)
                .where(ds_slice.isel(obs=-1).FT_WRB == False)
                .where(ds_slice.isel(obs=-1).F_RT_WSS).lat.data.flatten())
time =   (ds_slice.where(ds_slice.isel(obs=-1).FT_NFaroes == False)
                .where(ds_slice.isel(obs=-1).FT_WRB == False)
                .where(ds_slice.isel(obs=-1).F_RT_WSS).z.data.flatten())

In [ ]:
plot_tracks(lon,lat,time,'trajectories from Rockall Trough and Scottish Shelf and south to north through F-S Channel')

### Tracks starting at RT going north through FS Channel

In [ ]:
ds_slice = ds_RT_all_routes.isel(traj=slice(0,-1,100))
lon =    (ds_slice.where(ds_slice.isel(obs=-1).FT_NFaroes == False)
                .where(ds_slice.isel(obs=-1).FT_WRB == False)
                .where(ds_slice.isel(obs=-1).T_FS).lon.data.flatten())
lat =    (ds_slice.where(ds_slice.isel(obs=-1).FT_NFaroes == False)
                .where(ds_slice.isel(obs=-1).FT_WRB == False)
                .where(ds_slice.isel(obs=-1).T_FS).lat.data.flatten())
time =   (ds_slice.where(ds_slice.isel(obs=-1).FT_NFaroes == False)
                .where(ds_slice.isel(obs=-1).FT_WRB == False)
                .where(ds_slice.isel(obs=-1).T_FS).z.data.flatten())

In [ ]:
plot_tracks(lon,lat,time,'trajectories from Rockall Trough and Scottish Shelf and south to north through F-S Channel')

### Transport time series

In [ ]:
ParticleTransport_FS = (sectionAreaEW_FS*ds_FS_all.isel(obs=0).vvel*lat2metres_FS +
                            sectionAreaNS_FS*ds_FS_all.isel(obs=0).uvel*lon2metres_FS)/(nParticles_FS*1.0e06)


In [ ]:
y = ParticleTransport_FS*ds_FS_all_routes.F_RT_WSS*(1-ds_FS_all_routes.FT_NFaroes)*(1-ds_FS_all_routes.FT_WRB)

In [ ]:
display(ParticleTransport_FS*ds_FS_all_routes.F_RT_WSS)

In [ ]:
y.name =  'volume_transport'
y.attrs["units"]  = 'Sv'

fig,ax = plt.subplots(2,figsize = (12,5))

fig.suptitle('Time series of volume of water passing $57.5\degree$N reaching FS Channel on 2016-06-16')
ax[0].set_ylabel(y.name + ' ['+y.attrs["units"]+']')
ax[0].plot(ds_FS_all_routes.time.min("traj").data[:729],y.sum("traj")[:730].diff("obs"),alpha = 0.3)
ax[0].set_prop_cycle(None)
ax[0].plot(ds_FS_all_routes.time.min("traj").data[:729],
        y.sum("traj")[:730].diff("obs").rolling(obs=14,center = True).mean())


ax[1].set_ylabel('cumulative_'+y.name + '\n after time ['+y.attrs["units"]+']')
ax[1].plot(ds_FS_all_routes.time.min("traj").data[:730],y.sum("traj")[:730])

plt.show()

## Estimate transports through FSC

In [ ]:
transports()